<a href="https://colab.research.google.com/github/reynancs/study-notes/blob/main/notebooks/machine_learning/multiplas_abordagens_inter_coef_regressao_log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from scipy import stats

# =============================================================================
# 1. PREPARAÇÃO E TREINAMENTO DO MODELO
# =============================================================================

# Exemplo de treinamento (substitua pelos seus dados)
# X_train, y_train = seus dados de treino
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)

# Treinar o modelo
# model = LogisticRegression(random_state=42)
# model.fit(X_train_scaled, y_train)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from scipy import stats

# =============================================================================
# 1. PREPARAÇÃO E TREINAMENTO DO MODELO
# =============================================================================

# Exemplo de treinamento (substitua pelos seus dados)
# X_train, y_train = seus dados de treino
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)

# Treinar o modelo
# model = LogisticRegression(random_state=42)
# model.fit(X_train_scaled, y_train)

# =============================================================================
# 2. INTERPRETAÇÃO DOS COEFICIENTES - MÚLTIPLAS ABORDAGENS
# =============================================================================

def interpret_logistic_coefficients(model, feature_names, X_train_scaled=None):
    """
    Função completa para interpretação de coeficientes da Regressão Logística
    """

    # Extrair coeficientes
    coefficients = model.coef_[0]
    intercept = model.intercept_[0]

    # Criar DataFrame com interpretações
    results_df = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': coefficients,
        'Odds_Ratio': np.exp(coefficients),
        'Abs_Coefficient': np.abs(coefficients)
    })

    # Ordenar por importância (valor absoluto)
    results_df = results_df.sort_values('Abs_Coefficient', ascending=False)

    print("="*80)
    print("INTERPRETAÇÃO DOS COEFICIENTES - REGRESSÃO LOGÍSTICA")
    print("="*80)

    print(f"\n📊 INTERCEPT (β₀): {intercept:.4f}")
    print(f"   → Odds base (quando todas features = 0): {np.exp(intercept):.4f}")
    print(f"   → Probabilidade base: {1/(1+np.exp(-intercept)):.4f}")

    print(f"\n📈 TOP 10 FEATURES MAIS IMPORTANTES:")
    print("-"*60)
    for idx, row in results_df.head(10).iterrows():
        coef = row['Coefficient']
        odds_ratio = row['Odds_Ratio']
        feature = row['Feature']

        # Interpretação do sinal
        direction = "↗️ AUMENTA" if coef > 0 else "↘️ DIMINUI"

        # Interpretação do odds ratio
        if odds_ratio > 1:
            change = f"{((odds_ratio-1)*100):.1f}% mais chance"
        else:
            change = f"{((1-odds_ratio)*100):.1f}% menos chance"

        print(f"{feature:<25} | Coef: {coef:>8.4f} | OR: {odds_ratio:>6.3f}")
        print(f"{'':25} | {direction} churn | {change}")
        print("-"*60)

    return results_df

# =============================================================================
# 3. VISUALIZAÇÕES PARA INTERPRETAÇÃO
# =============================================================================

def plot_coefficient_analysis(results_df, top_n=15):
    """
    Criar visualizações para análise dos coeficientes
    """

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # 1. Coeficientes (valores brutos)
    top_features = results_df.head(top_n)
    colors = ['red' if x < 0 else 'green' for x in top_features['Coefficient']]

    axes[0,0].barh(range(len(top_features)), top_features['Coefficient'], color=colors, alpha=0.7)
    axes[0,0].set_yticks(range(len(top_features)))
    axes[0,0].set_yticklabels(top_features['Feature'])
    axes[0,0].set_xlabel('Coeficiente')
    axes[0,0].set_title('Coeficientes por Feature')
    axes[0,0].axvline(x=0, color='black', linestyle='--', alpha=0.5)
    axes[0,0].invert_yaxis()

    # 2. Odds Ratios
    axes[0,1].barh(range(len(top_features)), top_features['Odds_Ratio'],
                   color=['red' if x < 1 else 'green' for x in top_features['Odds_Ratio']], alpha=0.7)
    axes[0,1].set_yticks(range(len(top_features)))
    axes[0,1].set_yticklabels(top_features['Feature'])
    axes[0,1].set_xlabel('Odds Ratio')
    axes[0,1].set_title('Odds Ratio por Feature')
    axes[0,1].axvline(x=1, color='black', linestyle='--', alpha=0.5)
    axes[0,1].invert_yaxis()

    # 3. Importância por valor absoluto
    axes[1,0].barh(range(len(top_features)), top_features['Abs_Coefficient'],
                   color='blue', alpha=0.7)
    axes[1,0].set_yticks(range(len(top_features)))
    axes[1,0].set_yticklabels(top_features['Feature'])
    axes[1,0].set_xlabel('|Coeficiente|')
    axes[1,0].set_title('Importância das Features (Valor Absoluto)')
    axes[1,0].invert_yaxis()

    # 4. Distribuição dos coeficientes
    axes[1,1].hist(results_df['Coefficient'], bins=20, alpha=0.7, color='purple')
    axes[1,1].axvline(x=0, color='red', linestyle='--', alpha=0.7)
    axes[1,1].set_xlabel('Coeficiente')
    axes[1,1].set_ylabel('Frequência')
    axes[1,1].set_title('Distribuição dos Coeficientes')

    plt.tight_layout()
    plt.show()

# =============================================================================
# 4. ANÁLISE DE SIGNIFICÂNCIA ESTATÍSTICA
# =============================================================================

def statistical_significance_analysis(model, X_train_scaled, y_train, feature_names):
    """
    Análise de significância estatística dos coeficientes
    """
    from sklearn.metrics import log_loss

    # Predições
    y_pred_proba = model.predict_proba(X_train_scaled)[:, 1]

    # Calcular erros padrão aproximados
    # Usando aproximação: SE ≈ sqrt(diagonal da matriz de covariância)

    # Matriz de design
    X_with_intercept = np.column_stack([np.ones(X_train_scaled.shape[0]), X_train_scaled])

    # Matriz W (pesos)
    W = np.diag(y_pred_proba * (1 - y_pred_proba))

    # Matriz de covariância aproximada
    try:
        cov_matrix = np.linalg.inv(X_with_intercept.T @ W @ X_with_intercept)
        std_errors = np.sqrt(np.diag(cov_matrix))[1:]  # Excluir intercept

        # Z-scores e p-values
        z_scores = model.coef_[0] / std_errors
        p_values = 2 * (1 - stats.norm.cdf(np.abs(z_scores)))

        # Criar DataFrame
        significance_df = pd.DataFrame({
            'Feature': feature_names,
            'Coefficient': model.coef_[0],
            'Std_Error': std_errors,
            'Z_Score': z_scores,
            'P_Value': p_values,
            'Significant_05': p_values < 0.05,
            'Significant_01': p_values < 0.01
        })

        significance_df = significance_df.sort_values('P_Value')

        print("\n📊 ANÁLISE DE SIGNIFICÂNCIA ESTATÍSTICA")
        print("="*70)
        print(f"{'Feature':<25} | {'Coef':<8} | {'P-Value':<8} | {'Sig?'}")
        print("-"*70)

        for idx, row in significance_df.head(15).iterrows():
            sig_marker = "***" if row['P_Value'] < 0.001 else "**" if row['P_Value'] < 0.01 else "*" if row['P_Value'] < 0.05 else ""
            print(f"{row['Feature']:<25} | {row['Coefficient']:>7.4f} | {row['P_Value']:>7.4f} | {sig_marker}")

        return significance_df

    except np.linalg.LinAlgError:
        print("⚠️ Não foi possível calcular a significância (matriz singular)")
        return None

# =============================================================================
# 5. INTERPRETAÇÃO PRÁTICA PARA NEGÓCIO
# =============================================================================

def business_interpretation(results_df, top_n=10):
    """
    Interpretação voltada para o negócio
    """

    print("\n💼 INTERPRETAÇÃO PARA NEGÓCIO")
    print("="*80)

    # Features que mais aumentam o churn
    positive_coefs = results_df[results_df['Coefficient'] > 0].head(top_n//2)
    if len(positive_coefs) > 0:
        print("\n🔴 FATORES DE RISCO (Aumentam probabilidade de Churn):")
        print("-"*60)
        for idx, row in positive_coefs.iterrows():
            odds_increase = ((row['Odds_Ratio'] - 1) * 100)
            print(f"• {row['Feature']}")
            print(f"  → Aumenta as chances de churn em {odds_increase:.1f}%")
            print(f"  → Odds Ratio: {row['Odds_Ratio']:.3f}")
            print()

    # Features que mais diminuem o churn
    negative_coefs = results_df[results_df['Coefficient'] < 0].head(top_n//2)
    if len(negative_coefs) > 0:
        print("🟢 FATORES PROTETIVOS (Diminuem probabilidade de Churn):")
        print("-"*60)
        for idx, row in negative_coefs.iterrows():
            odds_decrease = ((1 - row['Odds_Ratio']) * 100)
            print(f"• {row['Feature']}")
            print(f"  → Diminui as chances de churn em {odds_decrease:.1f}%")
            print(f"  → Odds Ratio: {row['Odds_Ratio']:.3f}")
            print()

# =============================================================================
# 6. EXEMPLO DE USO COMPLETO
# =============================================================================

def complete_interpretation_example():
    """
    Exemplo completo de interpretação
    """

    print("="*80)
    print("EXEMPLO DE INTERPRETAÇÃO COMPLETA - REGRESSÃO LOGÍSTICA")
    print("="*80)

    # Exemplo com dados fictícios
    np.random.seed(42)
    feature_names = ['tenure', 'monthly_charges', 'total_charges', 'num_services',
                     'support_tickets', 'contract_length', 'payment_delay_days',
                     'satisfaction_score', 'usage_gb', 'discount_applied']

    # Coeficientes fictícios para exemplo
    example_coefficients = np.array([-0.8, 1.2, 0.3, -0.5, 0.9, -1.1, 0.7, -0.6, -0.4, -0.3])

    # Criar DataFrame de exemplo
    example_df = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': example_coefficients,
        'Odds_Ratio': np.exp(example_coefficients),
        'Abs_Coefficient': np.abs(example_coefficients)
    }).sort_values('Abs_Coefficient', ascending=False)

    print("\n📊 EXEMPLO DE INTERPRETAÇÃO:")
    print("-"*60)

    for idx, row in example_df.head(5).iterrows():
        coef = row['Coefficient']
        odds_ratio = row['Odds_Ratio']
        feature = row['Feature']

        if coef > 0:
            interpretation = f"Cada unidade ↑ em {feature} → {((odds_ratio-1)*100):.1f}% mais chance de churn"
        else:
            interpretation = f"Cada unidade ↑ em {feature} → {((1-odds_ratio)*100):.1f}% menos chance de churn"

        print(f"{feature}: {interpretation}")

    return example_df

# Executar exemplo
example_results = complete_interpretation_example()

print("""

💡 RESUMO DAS MELHORES PRÁTICAS:

1. **PADRONIZAÇÃO**: Sempre padronize as features antes do treino
2. **ODDS RATIO**: Use exp(coeficiente) para interpretação intuitiva
3. **SIGNIFICÂNCIA**: Verifique p-values para validar a importância
4. **VISUALIZAÇÃO**: Use gráficos para comunicar os insights
5. **CONTEXTO DE NEGÓCIO**: Traduza os números para ações práticas

📈 DICAS IMPORTANTES:
• Coeficiente positivo = aumenta chance de churn
• Coeficiente negativo = diminui chance de churn
• Odds Ratio > 1 = fator de risco
• Odds Ratio < 1 = fator protetor
• |Coeficiente| maior = maior impacto na predição
""")

EXEMPLO DE INTERPRETAÇÃO COMPLETA - REGRESSÃO LOGÍSTICA

📊 EXEMPLO DE INTERPRETAÇÃO:
------------------------------------------------------------
monthly_charges: Cada unidade ↑ em monthly_charges → 232.0% mais chance de churn
contract_length: Cada unidade ↑ em contract_length → 66.7% menos chance de churn
support_tickets: Cada unidade ↑ em support_tickets → 146.0% mais chance de churn
tenure: Cada unidade ↑ em tenure → 55.1% menos chance de churn
payment_delay_days: Cada unidade ↑ em payment_delay_days → 101.4% mais chance de churn


💡 RESUMO DAS MELHORES PRÁTICAS:

1. **PADRONIZAÇÃO**: Sempre padronize as features antes do treino
2. **ODDS RATIO**: Use exp(coeficiente) para interpretação intuitiva  
3. **SIGNIFICÂNCIA**: Verifique p-values para validar a importância
4. **VISUALIZAÇÃO**: Use gráficos para comunicar os insights
5. **CONTEXTO DE NEGÓCIO**: Traduza os números para ações práticas

📈 DICAS IMPORTANTES:
• Coeficiente positivo = aumenta chance de churn
• Coeficiente negativo